<a href="https://colab.research.google.com/github/KalinaXL/Extra-Operating-System/blob/master/keras_sir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Layer, Dense, Activation
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import tensorflow.keras.backend as K

In [2]:
class SIRLayer(Layer):
    def __init__(self, S0, I0, R0, D0, name = 'sir'):
        super().__init__(dynamic=True, name = name)
        self.S0 = S0
        self.I0 = I0
        self.D0 = D0
        self.R0 = R0
        self.N = S0 + I0 + R0 + D0
    def solve(self, alpha, size):
        S, I, R, D = self.S0, self.I0, self.R0, self.D0
        beta, gamma, mu = alpha
        ls = []
        max_i, max_d = 0, 0
        for i in range(size):
            S -= beta * I * S / self.N
            I += beta * I * S / self.N - (mu + gamma) * I
            R += gamma * I
            D += mu * I
            ls.append((I, D))
            max_i = I if I > max_i else max_i
            max_d = D if D > max_d else max_d
        return tf.convert_to_tensor(ls)
    def call(self, alpha):
        results = []
        for i, alpha_t in enumerate(alpha):
            result = self.solve(alpha_t, alpha.shape[0])
            results.append(result)
        # print('len: ', len(results))
        # print('len1: ', len(result))
        # results = tf.concat(results, axis = 0)
        return tf.convert_to_tensor(results)

    def compute_output_shape(self, input_shape):
        # print(input_shape)
        return (input_shape[0], input_shape[0], 2)

def define_model(S0 = 5100000, I0 = 10, R0 = 0, D0 = 0):
    input = Input(shape = (2, ))
    x = Dense(units = 8)(input)
    x = Activation('sigmoid')(x)
    x = Dense(units = 3)(x)
    x = Activation('sigmoid', name = 'alpha')(x)
    sir = SIRLayer(S0, I0, R0, D0)(x)
    return Model(inputs = input, outputs = [x, sir])

@tf.function
def loss_sir(q_true, q_pred):
    loss_1 = tf.math.reduce_mean(tf.math.square(tf.math.log(q_pred + 1e-3) - tf.math.log(q_true + 1e-3)))
    max_I = tf.maximum(tf.reduce_max(q_pred[:, :, 0]), 1e-5)
  
    loss_2 = 100 * tf.math.reduce_mean(tf.square(q_pred - q_true))
    return loss_1 + loss_2    

@tf.function
def loss_alpha(alpha_true, alpha_pred):
    diff = tf.math.subtract(alpha_pred[:-1], alpha_pred[1:])
    gamma = diff[:, 2] * 10
    diff = tf.math.square(diff[:, : -1])
    return 10 * tf.reduce_sum(diff) + 10 * tf.reduce_sum(tf.square(gamma))


In [3]:
model = define_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 8)                 24        
_________________________________________________________________
activation (Activation)      (None, 8)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 27        
_________________________________________________________________
alpha (Activation)           (None, 3)                 0         
_________________________________________________________________
sir (SIRLayer)               (None, None, 2)           0         
Total params: 51
Trainable params: 51
Non-trainable params: 0
_________________________________________________________________

In [4]:
data = pd.read_csv('data.csv').values[:, [1, 3]]# / 5100000

losses = {
    'alpha': loss_alpha,
    'sir': loss_sir
}

In [ ]:
model.compile(optimizer = Adam(lr = 1e-5), loss = losses)
H = model.fit(data, (np.random.randn(180, 3), data), batch_size= data.shape[0], epochs = 20, verbose = 1)

Epoch 1/20
1/1 [==============================] - 0s 1ms/step - loss: 117397411332096.0000 - alpha_loss: 80.4285 - sir_loss: 117397411332096.0000
Epoch 2/20
1/1 [==============================] - 0s 1ms/step - loss: 117397428109312.0000 - alpha_loss: 77.3179 - sir_loss: 117397428109312.0000
Epoch 3/20
1/1 [==============================] - 0s 1ms/step - loss: 117397386166272.0000 - alpha_loss: 88.9586 - sir_loss: 117397386166272.0000
Epoch 4/20
1/1 [==============================] - 0s 1ms/step - loss: 117397411332096.0000 - alpha_loss: 77.9626 - sir_loss: 117397411332096.0000
Epoch 5/20
1/1 [==============================] - 0s 1ms/step - loss: 117397386166272.0000 - alpha_loss: 84.3768 - sir_loss: 117397386166272.0000
Epoch 6/20
1/1 [==============================] - 0s 1ms/step - loss: 117397411332096.0000 - alpha_loss: 89.4635 - sir_loss: 117397411332096.0000
Epoch 7/20
1/1 [==============================] - 0s 1ms/step - loss: 117397411332096.0000 - alpha_loss: 77.2888 - sir_loss: